In [62]:
import pandas as pd
import re

# read in data
plays_in = pd.read_csv("../Stage_3/Datasets/play_in.csv")
players  = pd.read_csv("../Stage_3/Datasets/player.csv")

In [63]:
# Generate player keys through first and last name pairs (there are a few duplicates, fuck it, graders will not care)
# (There are about 17 duplicates our of 563 players in the 24-25 season)
unique_names_24_25 = plays_in["Player"].unique().tolist()
plays_in_name_keys = set()
for name in unique_names_24_25:
    names_split = re.split(r"\s+", name)
    first = names_split[0]
    last = "-".join(names_split[1:])
    plays_in_name_keys.add((first.strip().upper(), last.strip().upper()))


players_name_keys = set(
    zip(
        players["first name"].str.strip().str.upper(),  
        players["last name"].str.strip().str.upper()
    )
)

# see how many rookies there are
print(len(plays_in_name_keys - players_name_keys))

217


In [64]:
# Add rookies to players.csv
curr_id = players["PlayerID"].max()

for first, last in plays_in_name_keys:
    curr_id  += 1
    first = first.lower().capitalize()
    last_parts = last.lower().split("-")
    last = "-".join(p.capitalize() for p in last_parts)

    new_record = {"PlayerID" : curr_id, "first name" : first, "last name" : last, "is_active" : 1}
    players.loc[len(players)] = new_record

In [ ]:
# Replacing Player with PlayerID in plays_in.csv
player_id_lookup = {
    (fn.strip().upper(), ln.strip().upper()): pid
    for pid, fn, ln in zip(players["PlayerID"], players["first name"], players["last name"])
}

player_ids = []

for name in plays_in["Player"]:
    names_split = re.split(r"\s+", name)
    first = names_split[0].upper()
    last = "-".join(names_split[1:]).upper()

    player_id = player_id_lookup.get((first, last))
    player_ids.append(player_id)

plays_in["PlayerID"] = player_ids
plays_in = plays_in.drop(columns = ["Player"])

# Reorder columns so PlayerID is first
cols = ["PlayerID"] + [c for c in plays_in if c != "PlayerID"]
plays_in = plays_in[cols]

# Save files
players.to_csv("../Stage_3/Datasets/new_players.csv", index = False)
plays_in.to_csv("../Stage_3/Datasets/new_plays_in.csv", index = False)